# Car Renting - Modelling

## Autores:
- Alejandro De Haro
- Daniel Escobosa
- Pablo Berastegui
- Jose María Benitez

### Importing packages and modules

In [1]:
# module for building the pyomo model
import pyomo.environ as pe
# module for solving the pyomo model
import pyomo.opt as po

### Create the model

In [2]:
model = pe.ConcreteModel() 

Order to build the model:
1. Sets
1. Parameters
1. Variables
1. Objective function
1. Constraints

#### Sets

$car$: type of car $\{A,B,C\}$

$c$: country $\{\mathrm{ES, FR, DE, AT, CH, IT}\}$ (travel order)

In [ ]:
# Ordered trip sequence
country_seq = ['ES','FR','DE','AT','CH','IT']

# Pyomo sets
model.car = pe.Set(initialize=['A','B','C'])
model.country = pe.Set(initialize=country_seq)
model.country_after_first = pe.Set(initialize=country_seq[1:])

prev_map = {'FR':'ES','DE':'FR','AT':'DE','CH':'AT','IT':'CH'}
model.prev_country = pe.Param(model.country_after_first, initialize=prev_map)

'Any'. The default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend the
domain of this Paramto be 'Any', you can suppress this warning by explicitly
specifying 'within=Any' to the Param constructor.  (deprecated in 5.6.9, will
be removed in (or after) 6.0) (called from
/Users/descobosa/anaconda3/lib/python3.11/site-
packages/pyomo/core/base/indexed_component.py:718)


#### Parameters

$FC_{car,c}$: fuel (travel) cost of using car **car** in country **c** \[€\]

$FEE$: additional paperwork cost per **change of vehicle** between consecutive countries \[€\]

In [ ]:
# Fuel cost table from the statement (in €)
data = {
    ('A','ES'):160, ('A','FR'):210, ('A','DE'):180, ('A','AT'):110, ('A','CH'):85,  ('A','IT'):170,
    ('B','ES'):120, ('B','FR'):240, ('B','DE'):165, ('B','AT'):135, ('B','CH'):100, ('B','IT'):160,
    ('C','ES'):150, ('C','FR'):200, ('C','DE'):175, ('C','AT'):140, ('C','CH'):115, ('C','IT'):135,
}

model.FC = pe.Param(model.car, model.country, initialize=data)
FEE_value = 25  
model.FEE = pe.Param(initialize=FEE_value)

#### Variables

$u_{car,c} \in \{0,1\}$: 1 if car **car** is used in country **c**, 0 otherwise

$y_c \in \{0,1\}$: 1 if there is a **change of vehicle** when entering country **c** (for $c \ne \mathrm{ES}$)

In [ ]:
model.use = pe.Var(model.car, model.country, within=pe.Binary)
model.change = pe.Var(model.country_after_first, within=pe.Binary)

#### Objective Function

Minimize total cost = travel fuel cost + change fees

$$\min \sum_{c} \left( \sum_{car} FC_{car,c}\,u_{car,c} \right)\;+\;\sum_{c \ne ES} FEE \cdot y_c$$

In [6]:
def obj_rule(model):
    fuel_cost = sum(model.FC[car,c] * model.use[car,c] for car in model.car for c in model.country)
    change_cost = sum(model.FEE * model.change[c] for c in model.country_after_first)
    return fuel_cost + change_cost

model.total_cost = pe.Objective(rule=obj_rule, sense=pe.minimize)

#### Constraints

Constraint #1: **Use exactly one car per country**

$$\sum_{car} u_{car,c} = 1 \quad \forall c$$

In [7]:
model.one_car_per_country = pe.ConstraintList()
for c in model.country:
    model.one_car_per_country.add( sum(model.use[car,c] for car in model.car) == 1 )

Constraint #2: **Change indicator is activated when the chosen car differs from the previous country**

For every car and country $c \ne ES$:
$$u_{car,c} - u_{car,prev(c)} \le y_c$$

This forces $y_c=1$ whenever the new chosen car at $c$ wasn't chosen in the previous country. 

In [8]:
model.change_linking = pe.ConstraintList()
for c in model.country_after_first:
    prev_c = model.prev_country[c]
    for car in model.car:
        model.change_linking.add( model.use[car,c] - model.use[car,prev_c] <= model.change[c] )

#### Solver definition and solve statement

In [ ]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

Set parameter Username
Set parameter LicenseID to value 2704768
Academic license - for non-commercial use only - expires 2026-09-08
Read LP format model from file /var/folders/pr/nst7vndn0l10vqp0s73wclhr0000gn/T/tmp89jzk_dz.pyomo.lp
Reading time = 0.00 seconds
x1: 21 rows, 23 columns, 63 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 23 columns and 63 nonzeros
Model fingerprint: 0xdd5658cd
Variable types: 0 continuous, 23 integer (23 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 975.0000000
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 21 rows, 22 columns, 63 nonzeros
Variable types: 0 continuous, 22 integer (22 binary)

Root relaxatio

In [10]:
# Total optimal cost
print(round(pe.value(model.total_cost),2))

890.0


In [11]:
# Chosen car per country
for c in country_seq:
    chosen = [car for car in model.car if pe.value(model.use[car,c]) > 0.5][0]
    print(f'Country {c}: use car {chosen}')

Country ES: use car B
Country FR: use car A
Country DE: use car A
Country AT: use car A
Country CH: use car A
Country IT: use car C


In [12]:
# Number of changes and where they occur
num_changes = 0
for c in country_seq[1:]:
    if pe.value(model.change[c]) > 0.5:
        num_changes += 1
        print(f'Change occurs when entering {c}')
print(f'Total changes: {num_changes}')

Change occurs when entering FR
Change occurs when entering IT
Total changes: 2
